In [ ]:
from langchain_core.prompts import PromptTemplate

# from langchain_google_genai import GoogleGenerativeAI
from langchain_openai import OpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter

# Initialize the model
# llm = GoogleGenerativeAI(model="gemini-1.5-pro")
llm = OpenAI(temperature=0.9)

result = llm.invoke("Test input")
print(result)


# First chain generates a story
story_prompt = PromptTemplate.from_template(
    "Write a short -5 lines - story about {topic} in a bad mood."
)
story_chain = story_prompt | llm | StrOutputParser()

# Second chain analyzes the story
analysis_prompt = PromptTemplate.from_template(
    "Analyze the following story's mood:\n{story}"
)
analysis_chain = analysis_prompt | llm | StrOutputParser()

# Combine chains
story_with_analysis = story_chain | analysis_chain

# Run the combined chain
story_analysis = story_with_analysis.invoke({"topic": "a rainy day"})
print("\nAnalysis:", story_analysis)

# Using RunnablePassthrough.assign to preserve original data
enhanced_chain = RunnablePassthrough.assign(
    story=story_chain  # Add 'story' key with generated content
).assign(
    analysis=analysis_chain  # Add 'analysis' key with analysis of the story
)

# Execute the chain
result = enhanced_chain.invoke({"topic": "a rainy day"})
print(
    result.keys()
)  # Output: dict_keys(['topic', 'story', 'analysis']) # dict_keys(['topic', 'story', 'analysis'])
# For more control over the output structure, we could also construct dictionaries manually:

# Alternative approach using dictionary construction
manual_chain = (
    RunnablePassthrough()  # Pass through input
    | {
        "story": story_chain,  # Add story result
        "topic": itemgetter("topic"),  # Preserve original topic
    }
    | RunnablePassthrough().assign(  # Add analysis based on story
        analysis=analysis_chain
    )
)
result = manual_chain.invoke({"topic": "a rainy day"})
print(result.keys())  # Output: dict_keys(['story', 'topic', 'analysis'])

# We can simplify this with dictionary conversion using a LCEL shorthand:
# Simplified dictionary construction
simple_dict_chain = story_chain | {"analysis": analysis_chain}
result = simple_dict_chain.invoke({"topic": "a rainy day"})
print(result.keys())  # Output: dict_keys(['analysis', 'output'])

 for diagnostic messages is a test


The diagnostic message for this input would likely be something along the lines of: "Diagnostic results: No issues found in the input provided."

Analysis: 

The mood of this story is gloomy and pessimistic. The description of the day as dark and the use of words like "dreading," "irritable," and "annoyance" all contribute to a negative and unhappy mood. The constant rain and thunder add to the sense of dreariness and the feeling that everything is going wrong. Overall, the mood is one of sadness, frustration, and a general sense of negativity.
dict_keys(['topic', 'story', 'analysis'])
dict_keys(['story', 'topic', 'analysis'])
dict_keys(['analysis'])
